In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-recommendation-rucode/sample_submission.csv
/kaggle/input/anime-recommendation-rucode/user_ratings.csv
/kaggle/input/anime-recommendation-rucode/anime.csv


In [6]:
# !pip3 install implicit
import implicit
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

from IPython.display import display
from os.path import join
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from urllib.request import urlretrieve

In [7]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans

In [18]:
df_ss = pd.read_csv('/kaggle/input/anime-recommendation-rucode/sample_submission.csv')
df_anime = pd.read_csv('/kaggle/input/anime-recommendation-rucode/anime.csv')
df = pd.read_csv('/kaggle/input/anime-recommendation-rucode/user_ratings.csv')
# df_ss.head()
# df_anime.head()
# df.info()

In [9]:
df.head()

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,121,8,1,0
1,0,12815,10,1,3
2,0,3588,9,1,13
3,0,392,8,3,0
4,1,1575,8,2,25


In [19]:
df = df[df['rating'] != 0]

In [21]:
df[df['rating'] == 1]

,user_id,anime_id,rating,watching_status,watched_episodes
1180,7,32878,1,4,2
1748,13,713,1,2,1
1787,13,3759,1,6,1
1794,13,756,1,6,0
1806,13,11757,1,2,25
...,...,...,...,...,...
19079844,98587,1361,1,2,1
19079851,98587,6922,1,2,1
19080996,98589,16592,1,4,2
19081100,98589,14045,1,2,13


In [22]:
df_ss_new = df_ss.copy()
df_ss_new[['id_user', 'id_anime']] = df_ss_new['Id'].str.split(expand=True)
# df_ss_new['id_anime'] = df_ss_new['Id'].str.split()[0]
df_ss_new = df_ss_new.astype({"id_user": int, "id_anime": int})
df_ss_new.head(5)

,Id,rating,id_user,id_anime
0,0 249,7,0,249
1,0 4181,4,0,4181
2,0 813,3,0,813
3,0 2167,6,0,2167
4,1 1838,3,1,1838


In [ ]:
# resplit
# df_resplit = df_ss.copy()
# df_resplit['Id']= df_ss_new['id_user'] + ' ' + df_ss_new['id_anime']
# df_resplit

### Trying something with SVD

In [23]:
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'anime_id', 'rating']], reader)

In [24]:
# algo = SVD(n_factors = 100,n_epochs = 1000,reg_all = 0.1)
svd = SVD(verbose=True, n_epochs = 30)
algo = KNNWithMeans(k=50, bsl_options = {'method': 'sgd', 'n_epochs': 20}, sim_options={'name': 'pearson_baseline', 'user_based': False}, verbose=True)

In [ ]:
# cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

In [25]:
trainset = data.build_full_trainset()

In [ ]:
# https://kerpanic.wordpress.com/2018/03/26/a-gentle-guide-to-recommender-systems-with-surprise/
algo.fit(trainset)

In [26]:
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [ ]:
svd.predict(0, 0).est

In [27]:
rating__ = np.full((876529, 1), 0.0)
for i, row in df_ss_new.iterrows():
#     print(row.id_user, ' ', row.id_anime, ' ', svd.predict(row.id_user, row.id_anime).est)
    rating__[i] = svd.predict(row.id_user, row.id_anime).est

In [28]:
df_ss_new['rating'] = rating__

In [29]:
df_ss_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876529 entries, 0 to 876528
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Id        876529 non-null  object 
 1   rating    876529 non-null  float64
 2   id_user   876529 non-null  int64  
 3   id_anime  876529 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 26.7+ MB


In [ ]:
# test_data = Dataset.load_from_df(df_ss_new[['id_user', 'id_anime', 'rating']], reader)
# svd.test()
# svd.predict(24,4155).est
df_ss_new.head(10)

In [32]:
df_ss_new = df_ss_new.drop(columns=['id_user', 'id_anime'])
df_ss_new

KeyError: "['id_user' 'id_anime'] not found in axis"

In [31]:
df_ss_new.to_csv('submit2.csv', index=False)